In [17]:
import os
from mayavi import mlab
from mayavi.sources.api import VTKFileReader
from mayavi.sources.vtk_data_source import VTKDataSource
from mayavi.core.api import Engine
from mayavi.scripts import mayavi2
import numpy as np
from tvtk.api import tvtk
from mayavi.modules.surface import Surface
from vtk import *

In [18]:
filename=os.path.join(os.getcwd(),'cavity/VTK/cavity_100.vtk')

In [19]:
# read the source file
mlab.init_notebook(local=True,width=500,height=500)
fig = mlab.figure()

Notebook initialized with x3d backend.


In [20]:
reader = vtkUnstructuredGridReader()
reader.SetFileName(filename)
reader.Update()

In [21]:
data = reader.GetOutput()
#print(data)

In [22]:
points = data.GetPoints()
points_data = data.GetPointData()
pressure = points_data.GetArray(0)
velocity = points_data.GetArray(1)

In [23]:
from vtk.util import numpy_support
np_points = numpy_support.vtk_to_numpy(points.GetData())
np_pressure = numpy_support.vtk_to_numpy(pressure)
np_velocity = numpy_support.vtk_to_numpy(velocity)
order = np.array([np.arange(0,882,1)])
order = np.transpose(order)
new_col = np.hstack((order,np_points))
#m = mlab.points3d(np_points[:,0],np_points[:,1],np_points[:,2],np_velocity[:,0])
q = mlab.quiver3d(np_points[:441,0],np_points[:441,1],np_points[:441,2],np_velocity[:441,0],np_velocity[:441,1],np_velocity[:441,2])

In [24]:
fig